objectives
- find the top hit for each contig in the mmseqs output, across all dbs
- determine what percentage of contigs were classified
- determine what percentage of reads were classified

find the top hit for each contig in the blast output

In [1]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "StatsBase",
    "ProgressMeter",
    "uCSV"
]
# Pkg.activate(; temp=true)
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [ ]:
# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

In [2]:
# NCBI host metadata
ncbi_metadata_file = joinpath(dirname(pwd()), "metadata", "NCBI-virus-refseq.transformed.tsv")
ncbi_host_metadata = DataFrames.DataFrame(uCSV.read(ncbi_metadata_file, header=1, delim='\t', encodings=Dict("false" => false, "true" => true)))

# ICTV host metadata
ictv_metadata_file = joinpath(dirname(pwd()), "metadata", "VMR_MSL38_v1 - VMR MSL38 v1.transformed.tsv")
ictv_host_metadata = DataFrames.DataFrame(uCSV.read(ictv_metadata_file, header=1, delim='\t', typedetectrows=100))
ictv_host_metadata = ictv_host_metadata[.!isempty.(ictv_host_metadata[!, "taxid"]), :]
ictv_host_metadata[!, "taxid"] = parse.(Int, ictv_host_metadata[!, "taxid"])

# # VirusHostDB metadata
virushostdb_metadata_file = joinpath(dirname(pwd()), "metadata", "virushostdb.transformed.tsv")
virushostdb_metadata = DataFrames.DataFrame(uCSV.read(virushostdb_metadata_file, header=1, delim='\t', typedetectrows=1086, encodings=Dict("missing" => missing, "false" => false, "true" => true)))

vertebrate_taxids = union(
    ictv_host_metadata[ictv_host_metadata[!, "Host source"] .== "vertebrates", "taxid"],
    ncbi_host_metadata[ncbi_host_metadata[!, "host_is_vertebrate"], "taxid"],
    virushostdb_metadata[virushostdb_metadata[!, "host_is_vertebrate"], "virus_taxid"]
)

human_taxids = union(
    ncbi_host_metadata[ncbi_host_metadata[!, "host_is_human"], "taxid"],
    virushostdb_metadata[virushostdb_metadata[!, "host_is_human"], "virus_taxid"]
)

┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213


1559-element Vector{Int64}:
  351073
 3051992
 3052643
 3052759
 2795181
 2304647
 2825847
   11029
 3052040
 2845899
 2055263
   95342
 1965018
       ⋮
  356663
  356664
  373193
   11837
   38804
  132475
   11089
  617102
 2053026
  186538
 1764086
   64320

In [3]:
base_dir = dirname(pwd())
data_dir = joinpath(base_dir, "data")
sample_directories = readdir(joinpath(data_dir, "SRA"), join=true)

594-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399459"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399460"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399461"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399462"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399463"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399464"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399465"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399466"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399467"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399468"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399469"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/

In [4]:
easy_taxonomy_lca_reports = String[]
for sample_directory in sample_directories
    mmseqs_directory_contents = readdir(joinpath(sample_directory, "mmseqs_easy_taxonomy"), join=true)
    sample_lca_reports = filter(x -> occursin("final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.", x) && occursin("_lca.tsv", x), mmseqs_directory_contents)
    append!(easy_taxonomy_lca_reports, sample_lca_reports)
end
easy_taxonomy_lca_reports

1782-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 128 bytes ⋯ "asy_taxonomy.UniRef100_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef50_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef90_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 128 bytes ⋯ "asy_taxonomy.UniRef100_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef50_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef90_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 128 bytes ⋯ "asy_taxonomy.UniRef100_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef50_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef90_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 128 bytes ⋯ "asy_taxonomy.UniRef100_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRef50_lca.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 127 bytes ⋯ "easy_taxonomy.UniRe

In [5]:
lca_tsv = first(easy_taxonomy_lca_reports)

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy/final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.UniRef100_lca.tsv"

In [6]:
joint_lca_table = DataFrames.DataFrame()
ProgressMeter.@showprogress for lca_tsv in easy_taxonomy_lca_reports
    method = replace(replace(basename(lca_tsv), "final.contigs.fastg.gfa.fna." => ""), "_lca.tsv" => "")
    # @show method
    this_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(lca_tsv)
    this_lca_table[!, "sample"] .= basename(dirname(dirname(lca_tsv)))
    this_lca_table[!, "method"] .= method
    append!(joint_lca_table, this_lca_table)
end
joint_lca_table

Progress: 100%|█████████████████████████████████████████| Time: 0:05:19


Row,contig_id,taxon_id,taxon_rank,taxon_name,fragments_retained,fragments_taxonomically_assigned,fragments_in_agreement_with_assignment,support -log(E-value),sample,method
,Int64,Int64,String,String,Int64,Int64,Int64,Float64,String,String
1,6611,147550,class,Sordariomycetes,1,1,1,1.0,SRR6399459,mmseqs_easy_taxonomy.UniRef100
2,6612,229535,species,Penicillium nordicum,1,1,1,1.0,SRR6399459,mmseqs_easy_taxonomy.UniRef100
3,6617,470,species,Acinetobacter baumannii,1,1,1,1.0,SRR6399459,mmseqs_easy_taxonomy.UniRef100
4,6620,9597,species,Pan paniscus,3,2,1,0.61,SRR6399459,mmseqs_easy_taxonomy.UniRef100
5,6621,469,genus,Acinetobacter,1,1,1,1.0,SRR6399459,mmseqs_easy_taxonomy.UniRef100
6,6622,286661,species,Peltaster fructicola,3,3,1,0.91,SRR6399459,mmseqs_easy_taxonomy.UniRef100
7,6623,207598,subfamily,Homininae,5,5,3,0.61,SRR6399459,mmseqs_easy_taxonomy.UniRef100
8,6625,5139,order,Sordariales,1,1,1,1.0,SRR6399459,mmseqs_easy_taxonomy.UniRef100
9,6626,1,no rank,root,5,4,4,1.0,SRR6399459,mmseqs_easy_taxonomy.UniRef100


In [7]:
filtered_lca_table = joint_lca_table[map(x -> x in human_taxids, joint_lca_table[!, "taxon_id"]), :]

Row,contig_id,taxon_id,taxon_rank,taxon_name,fragments_retained,fragments_taxonomically_assigned,fragments_in_agreement_with_assignment,support -log(E-value),sample,method
,Int64,Int64,String,String,Int64,Int64,Int64,Float64,String,String
1,11702,61673,no rank,Porcine endogenous retrovirus,2,2,1,0.95,SRR6399469,mmseqs_easy_taxonomy.UniRef100
2,11702,61673,no rank,Porcine endogenous retrovirus,2,2,1,0.95,SRR6399469,mmseqs_easy_taxonomy.UniRef90
3,12729,61673,no rank,Porcine endogenous retrovirus,2,2,1,0.55,SRR6399478,mmseqs_easy_taxonomy.UniRef100
4,12729,61673,no rank,Porcine endogenous retrovirus,2,2,1,0.55,SRR6399478,mmseqs_easy_taxonomy.UniRef90
5,27515,1862825,no rank,Gemycircularvirus HV-GcV2,3,3,2,0.6,SRR6399485,mmseqs_easy_taxonomy.UniRef100
6,27368,1862825,no rank,Gemycircularvirus HV-GcV2,1,1,1,1.0,SRR6399485,mmseqs_easy_taxonomy.UniRef100
7,6298,1862825,no rank,Gemycircularvirus HV-GcV2,1,1,1,1.0,SRR6399485,mmseqs_easy_taxonomy.UniRef100
8,3075,1862825,no rank,Gemycircularvirus HV-GcV2,3,3,3,1.0,SRR6399485,mmseqs_easy_taxonomy.UniRef100
9,27683,1862825,no rank,Gemycircularvirus HV-GcV2,1,1,1,1.0,SRR6399485,mmseqs_easy_taxonomy.UniRef100


In [8]:
taxon_counts = collect(StatsBase.countmap(filtered_lca_table[!, "taxon_name"]))
taxon_counts = DataFrames.DataFrame(taxon_name = first.(taxon_counts), counts = last.(taxon_counts))
sort!(taxon_counts, "counts", rev=true)

Row,taxon_name,counts
,String,Int64
1,Gemycircularvirus HV-GcV2,282
2,Porcine endogenous retrovirus,123
3,Human endogenous retrovirus K,12
4,Human immunodeficiency virus 1,7
5,Human papillomavirus type 128,6
6,Human papillomavirus,5
7,Human papillomavirus 65,4
8,Human papillomavirus 23,4
9,Merkel cell polyomavirus,3


In [11]:
filtered_lca_table[map(x -> x == "Human immunodeficiency virus 1", filtered_lca_table[!, "taxon_name"]), :]

Row,contig_id,taxon_id,taxon_rank,taxon_name,fragments_retained,fragments_taxonomically_assigned,fragments_in_agreement_with_assignment,support -log(E-value),sample,method
,Int64,Int64,String,String,Int64,Int64,Int64,Float64,String,String
1,106241,11676,species,Human immunodeficiency virus 1,1,1,1,1.0,SRR6399695,mmseqs_easy_taxonomy.UniRef100
2,129207,11676,species,Human immunodeficiency virus 1,2,1,1,1.0,SRR6399727,mmseqs_easy_taxonomy.UniRef100
3,107543,11676,species,Human immunodeficiency virus 1,1,1,1,1.0,SRR6399772,mmseqs_easy_taxonomy.UniRef100
4,107543,11676,species,Human immunodeficiency virus 1,1,1,1,1.0,SRR6399772,mmseqs_easy_taxonomy.UniRef50
5,107543,11676,species,Human immunodeficiency virus 1,1,1,1,1.0,SRR6399772,mmseqs_easy_taxonomy.UniRef90
6,28979,11676,species,Human immunodeficiency virus 1,2,1,1,1.0,SRR6399977,mmseqs_easy_taxonomy.UniRef100
7,71619,11676,species,Human immunodeficiency virus 1,1,1,1,1.0,SRR6399994,mmseqs_easy_taxonomy.UniRef90


In [9]:
mmseqs_taxid_hits = unique(filtered_lca_table[!, "taxon_id"])

14-element Vector{Int64}:
   61673
 1862825
   45617
 1647924
   10566
  931209
  493803
   11676
   28312
   37955
  765052
  463676
 3052230
 1891726

In [ ]:
# kraken_taxid_hits = [
# 10566,
# 1175849,
# 2017081,
# 2025360,
# 2734294,
# 277944,
# 2955291,
# 2955746,
# 3050300,
# 3052399,
# 36427,
# 59304,
# 642022,
# 93678,
# 11137,
# 11676,
# 1239573,
# 1978540,
# 2304647,
# 290028,
# 3050298,
# 694009,
# 10243,
# 10245,
# 1513260,
# 1513263,
# 1961678,
# 3052317,
# 3052458,
# 3052499,
# 568715,
# 129875,
# 1678143,
# 1780507,
# 1891764,
# 1961681,
# 1987017,
# 291484,
# 1961680,
# 3048383,
# 1904876,
# 3050294,
# 1513256,
# 2200830,
# 393049,
# 129951,
# 130308,
# 2055263,
# 68887,
# 204269,
# 2049444,
# 2825847,
# 38170,
# 10244,
# 1962300,
# 944645,
# 1972576,
# 3048384,
# 45617,
# 2170195,
# 3050295,
# 1513257,
# 2025396,
# 3047956,
# 2844583,
# 1513258,
# 11983,
# 3050297,
# 3052413,
# 3050299,
# 10255,
# 2844646,
# 2021738,
# 2170197,
# 2956268,
# 1518574,
# 3050296,
# 130310,
# 1647924,
# 99000,
# 1511805,
# 10258,
# 3047957,
# 1891726,
# 38804,
# 108098,
# 1335626,
# 1922246,
# 2844585,
# ];

In [ ]:
# for x in unique(filtered_lca_table[map(x -> x in intersect(kraken_taxid_hits, mmseqs_taxid_hits), filtered_lca_table[!, "taxon_id"]), "taxon_name"])
#     println(x)
# end

In [ ]:
# Human endogenous retrovirus K
# Human papillomavirus KC5
# Human papillomavirus
# Human immunodeficiency virus 1
# Alphapolyomavirus quintihominis